In [7]:
import pandas as pd
import sys
# from google.colab import files
# import io 
import re
import datetime
import ast

from sklearn.model_selection import train_test_split

df = pd.read_csv("Kickstarter.csv")

"""
The dataset can be found here:
https://webrobots.io/kickstarter-datasets/
https://s3.amazonaws.com/weruns/forfun/Kickstarter/Kickstarter_2021-06-17T03_20_03_179Z.zip
"""

'\nThe dataset can be found here:\nhttps://webrobots.io/kickstarter-datasets/\nhttps://s3.amazonaws.com/weruns/forfun/Kickstarter/Kickstarter_2021-06-17T03_20_03_179Z.zip\n'

In [8]:
df = df.drop(columns=["currency_symbol",
                 "country_displayable_name",
                 "created_at",
                 "creator",
                 "country",
                 "urls",
                 "currency_trailing_code",
                 "friends",
                 "fx_rate",
                 "id",
                 "is_starred",
                 "name",
                 "permissions"], axis = 1)

In [9]:
df.head(3)

,backers_count,blurb,category,converted_pledged_amount,currency,current_currency,deadline,disable_communication,goal,is_backing,...,slug,source_url,spotlight,staff_pick,state,state_changed_at,static_usd_rate,usd_exchange_rate,usd_pledged,usd_type
0,65,A super-cheeky strategy card game that's full ...,"{""id"":34,""name"":""Tabletop Games"",""analytics_na...",2856,AUD,USD,1626228004,False,15000.0,NaN,...,zaka,https://www.kickstarter.com/discover/categorie...,False,False,live,1623636005,0.770798,0.762655,2886.886851,international
1,146,Help us finish a feature film about two strang...,"{""id"":31,""name"":""Narrative Film"",""analytics_na...",10120,USD,USD,1333349940,False,10000.0,NaN,...,help-us-finish-elevator-a-feature-length-film,https://www.kickstarter.com/discover/categorie...,True,False,successful,1333349940,1.000000,1.000000,10120.000000,international
2,41,Considered playfully mischievous and outlaws i...,"{""id"":254,""name"":""Performances"",""analytics_nam...",3785,USD,USD,1455388491,False,3000.0,NaN,...,phase,https://www.kickstarter.com/discover/categorie...,True,False,successful,1455388491,1.000000,1.000000,3785.000000,international


Do we want to combine find the length of the campaign from (created at and deadline) to see if there is value there. 

keep location, clean and ckeep city name



In [10]:
def clean_data(text):
    email_regex = r"\S*@\S*\s?"
    non_alpha = "[^a-zA-Z0-9 ]"
    multi_white_spaces = "[ ]{2,}"
    text = re.sub(email_regex, "", text)
    text = re.sub(non_alpha, "", text)
    text = re.sub(multi_white_spaces, "", text)
    return text.lower().lstrip().rstrip()

# Load The Dataset

In [11]:
# file_path = 'Downloads/Kickstarter_2021-06-17T03_20_03_179Z/Kickstarter001.csv'
file_path = "Kickstarter.csv"
# file_path_small_test = 'small_file.csv'


def clean_data(file_path):
    df = pd.read_csv(file_path)
    
    def change_to_time_series(item):
        item = datetime.datetime.fromtimestamp(item)
        return item
    
    # change the 'created-at', 'deadline', 'state_changed_at' columns
    # to time series columns
    time_series_columns = ['created_at', 'deadline', 'state_changed_at', 'launched_at']
    for column in time_series_columns:
        df[column] = df[column].apply(change_to_time_series)
    
    # dropping columns with all null values
    # for reference these the column names:
    # ['friends', 'is_backing', 'is_starred', 'permissions']
    all_null_columns = df.isnull().sum() == len(df)
    to_drop = all_null_columns[all_null_columns == True].index
    df = df.drop(columns=to_drop, axis=1)
    
    df = df.drop(columns=["currency_symbol",
                          "country_displayable_name",
                          "creator",
                          "country",
                          "urls",
                          "source_url",
                          "currency_trailing_code",
                          "fx_rate",
                          "id",
                          "name",
                          "disable_communication",
                          "photo", # has url for photos might be useful later 
                          "usd_type", 
                          'converted_pledged_amount',
                          'pledged',
                          'usd_exchange_rate',
                          'static_usd_rate',
                          'currency', # dropping due to have pledged in USD
                          'current_currency'  # dropping due to have pledged in USD
                         ], axis=1)
    
    def extract_dictionary_info(item):
        """ Use the apply method with the column name.
        Takes in dictionary in string form,
        converts it into a dictionary, and
        returns info_type requested in a new column.
        """
        my_dic = ast.literal_eval(item)
        return my_dic

    category_df = df['category'].apply(extract_dictionary_info).apply(pd.Series)
    category_df['parent_name'].loc[(category_df.parent_name.isnull() == True)& (category_df['name'] == 'Dance')] = 'Dance'
    category_df['parent_name'].loc[(category_df.parent_name.isnull() == True)& (category_df['name'] == 'Photography')] = 'Photography'
    category_df['parent_name'].loc[(category_df.parent_name.isnull() == True)& (category_df['name'] == 'Games')] = 'Games'
    
    df['category_2'] = category_df.parent_name
    df.drop(columns='category', inplace=True)
    # still need to remove the Nan's into the missing categories:
    # categories are 'Dance', 'Games', 'Photography'
    # taken from the 'name' in category
    
    # columns I can't get to work
    # in dictionary format need to work on OR text data
    # text_data need to vectorize: 'blurb
    drop_for_now = ['blurb', 'location', 'profile', 'slug']
    df.drop(columns=drop_for_now, inplace=True)
    
    
    # creating our target variable, we had the four sub-categories:
    # successful, failed, canceled, live
    # turn it into a binary variable
    df['target'] = 0
    df['target'].loc[df.state == 'successful'] = 1
    df.drop(columns='state', inplace=True)
    
    # changing boolean to integers
    df['spotlight_2'] = 0
    df['spotlight_2'].loc[df.spotlight == True] = 1

    df['staff_pick_2'] = 0
    df['staff_pick_2'].loc[df.staff_pick == True] = 1

    df['is_starrable_2'] = 0
    df['is_starrable_2'].loc[df.is_starrable == True] = 1
    df.drop(columns=['spotlight', 'staff_pick', 'is_starrable'],
           inplace=True)
    
    # removing the launched_at and state_changed_at time series columns
    # add at your discretion
    time_series = ['launched_at', 'state_changed_at']
    df.drop(columns=time_series, inplace=True)
    
    
    return df

In [12]:
# Reading in the data and passing through the cleaning function
df = clean_data(file_path)
# df = clean_data(file_path_small_test)
df.head(3)


C:\Users\iankn\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\Users\iankn\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\Users\iankn\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: 

,backers_count,created_at,deadline,friends,goal,is_backing,is_starred,permissions,usd_pledged,category_2,target,spotlight_2,staff_pick_2,is_starrable_2
0,65,2021-05-20 18:00:51,2021-07-13 19:00:04,NaN,15000.0,NaN,NaN,NaN,2886.886851,Games,0,0,0,1
1,146,2012-01-26 11:20:01,2012-04-01 23:59:00,NaN,10000.0,NaN,NaN,NaN,10120.000000,Film & Video,1,1,0,0
2,41,2015-11-13 14:04:33,2016-02-13 11:34:51,NaN,3000.0,NaN,NaN,NaN,3785.000000,Dance,1,1,0,0


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3662 entries, 0 to 3661
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   backers_count   3662 non-null   int64         
 1   created_at      3662 non-null   datetime64[ns]
 2   deadline        3662 non-null   datetime64[ns]
 3   friends         5 non-null      object        
 4   goal            3662 non-null   float64       
 5   is_backing      5 non-null      object        
 6   is_starred      5 non-null      object        
 7   permissions     5 non-null      object        
 8   usd_pledged     3662 non-null   float64       
 9   category_2      3662 non-null   object        
 10  target          3662 non-null   int64         
 11  spotlight_2     3662 non-null   int64         
 12  staff_pick_2    3662 non-null   int64         
 13  is_starrable_2  3662 non-null   int64         
dtypes: datetime64[ns](2), float64(2), int64(5), object(5)
me

In [14]:
# Encode these later for optimization
df["category_2"].value_counts()

Film & Video    1115
Dance           1109
Games            873
Fashion          565
Name: category_2, dtype: int64

In [15]:
# Drop all columns except these
# we will clean this up Tuesday
df = df.drop(df.columns.difference(["goal", "usd_pledged", "target", "is_starrable_2", "staff_pick_2",
                                    "spotlight_2","backers_count"]), axis = 1)

# Split the Data

In [16]:
# Split the data to avoid leakage
x = df.drop(columns='target')
y = df['target']

In [17]:
# Create a training and validation set
# Smaller test size is acceptable due to the amount of data available
X_train, X_val, y_train, y_val = train_test_split(x, y, test_size=.1, random_state=42)

print('x_train:', len(X_train))
print('x_val:', len(X_val))
print('y_train:',y_train.shape)
print('y_val:', y_val.shape)

x_train: 3295
x_val: 367
y_train: (3295,)
y_val: (367,)


# Explore Models

In [18]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, ReLU
from tensorflow.keras.callbacks import TensorBoard

from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# required for compatibility bewteen sklearn and keras
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

In [19]:
# Define input dimensions to avoid hidden layer issues
input_dims = X_train.shape[1]
input_dims

6

## Create Model Function ##
This function should help you save time on creating more models.

In [20]:
# Function to create model, required for KerasClassifier
def create_model(units = 128, optimizer = "adam", activation = "sigmoid"):
    """"
    Returns a complied keras model 
    
    Parameters
    ----------
    units: int 
        number of neruons/nodes/units to use in each hidden layer
        
    Returns
    -------
    model: keras object 
    """

    model = Sequential()
    model.add(Dense(units = 64, input_dim = input_dims, activation = activation))
    model.add(Dense(units = 32, activation = activation))
    model.add(Dense(units = 10, activation = activation))   
    model.add(Dense(1, activation="sigmoid")) # USE SIGMOID FOR BINARY CLASSIFICATION
    model.compile(loss="binary_crossentropy",
                    optimizer = "adam",
                    metrics=["accuracy"])
    return model

In [21]:
# Instantiate a base model for grid search
base_model = KerasClassifier(build_fn = create_model)

## Perform a Grid Search To Optimize Our Model ##

In [22]:
# define the grid search parameters
param_grid = {'batch_size': [32],
              'epochs': [3],
              'units':[32, 64]}

In [23]:
# Create Grid Search
grid = GridSearchCV(estimator = base_model,
                    param_grid = param_grid, 
                    n_jobs=-2, 
                    verbose=1, 
                    cv=3)
grid_result = grid.fit(X_train, y_train)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Fitting 3 folds for each of 2 candidates, totalling 6 fits
Epoch 1/3
103/103 [==============================] - 1s 875us/step - loss: 0.5599 - accuracy: 0.6938
Epoch 2/3
103/103 [==============================] - 0s 775us/step - loss: 0.3499 - accuracy: 0.8971
Epoch 3/3
103/103 [==============================] - 0s 784us/step - loss: 0.2452 - accuracy: 0.9551
Best: 0.9566023747126261 using {'batch_size': 32, 'epochs': 3, 'units': 64}
Means: 0.9556916356086731, Stdev: 0.004344004127878566 with: {'batch_size': 32, 'epochs': 3, 'units': 32}
Means: 0.9566023747126261, Stdev: 0.005047693096155494 with: {'batch_size': 32, 'epochs': 3, 'units': 64}


In [24]:
# Assign the best model 
best_model = grid_result.best_estimator_
# Confirm the models params
best_model.get_params()

{'batch_size': 32,
 'epochs': 3,
 'units': 64,
 'build_fn': <function __main__.create_model(units=128, optimizer='adam', activation='sigmoid')>}

In [25]:
    # Create a new model based on our best parameters
    base_model = Sequential()
    base_model.add(Dense(units = 64, input_dim = input_dims, activation = "sigmoid"))
    base_model.add(Dense(units = 32, activation = "sigmoid"))
    base_model.add(Dense(units = 10, activation = "sigmoid"))   
    base_model.add(Dense(1, activation="sigmoid"))

    base_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    base_model.fit(X_train,
              y_train,
              epochs=3, 
              validation_data = (X_val,y_val),
              verbose=2)

Epoch 1/3
103/103 - 1s - loss: 0.6370 - accuracy: 0.6795 - val_loss: 0.4766 - val_accuracy: 0.9537
Epoch 2/3
103/103 - 0s - loss: 0.3664 - accuracy: 0.9514 - val_loss: 0.2799 - val_accuracy: 0.9591
Epoch 3/3
103/103 - 0s - loss: 0.2501 - accuracy: 0.9554 - val_loss: 0.2231 - val_accuracy: 0.9591


In [26]:
# Save the entire model as a SavedModel.
!mkdir -p saved_model
base_model.save('saved_model/base_model') 

INFO:tensorflow:Assets written to: saved_model/base_model\assets
